# Exercise 4.1: The expression for the period $T$ of a *realistic pendulum*, for which the maximum angle of displacement is $\theta_0$, when $\theta_0$ is not necessarily small, is given by:

$T = \frac{T_0}{\pi} \int_0^{\theta_0} \frac{ \mathrm{d} \theta } { [\sin^2 (\theta_0/2) - \sin^2 (\theta/2)]^{1/2}}$,

where $T_0$ is the period for small-angle oscillations. 

By expanding the denominator and integrating it term-by-term, we can obtain a series expansion for the period:

$T \simeq T_0 \left[ 1 + \left(\frac{1}{2}\right)^2 \sin^2 \frac{\theta_0}{2} + \left( \frac{1 \cdot 3}{2 \cdot 4}\right)^2 \sin^4 \frac{\theta_0}{2} + \left( \frac{1 \cdot 3 \cdot 5}{2 \cdot 4 \cdot 6}\right)^2 \sin^6 \frac{\theta_0}{2} ...\right]$.

The expansion tells us e.g. that an amplitude $\theta_0 = 80^\circ$ leads to a 10\% slowdown of the pendulum relative to the small $\theta$ result. 

We will determine the period computationally, without the need for any expansions. 

(a) Use numerical quadrature to determine the ratio $T/T_0$ for six values of $\theta_0$ in the interval $[0,\pi/2]$. Show that you have attained at least four places of accuracy by progressively increasing the number of integration points until changes occur only in the fifth place or beyond. 

(b) Use the power series to determine $T/T_0$. Continue summing terms until changes in the sum occur only in the fifth place, or beyond. 

(c) Plot the values you obtained for $T/T_0$ vs. $\theta_0$ for both the integral and power series solution. 

BONUS 10%:

(d) Use Scipy's ```scipy.integrate.quad``` to perform the integral at the same values. Plot these as well as a third answer.

## Solution(a)

We make use of the ```gauss``` integrator from the lecture.

In [20]:
import math
import scipy # we need scipy for the N-th order Legendre polynomials.
from math import pi
from functools import partial

thetaList = [pi/12,pi/8,pi/6,pi/4,pi/3,pi/2] # large angles from from which T/T0 will be calculated

# Let's write a higher-order function that implements N-th order Gaussian quadrature:
def gauss(func, a, b, N):
    """Calculates the numerical integral of a function in the interval a,b using N-th order Gaussian quadrature"""
    # N-th order Gaussian quadrature 
    # get the weights and points from the scipy special function:
    xi, wi = scipy.special.roots_legendre(int(N))
    # now use the general formula to get the integral:
    integral = 0
    for j, wj in enumerate(wi):
        # calculate the x value using the xi array:
        xj = 0.5*((b-a) * xi[j] + b + a) # transform the xi[j] to the function argument, xj
        integral = integral + wj * func(xj)
    return (b-a)/2 * integral

def f(theta0,theta): # when f is integrated with respect to theta from 0 to theta0, the ratio T/T0 will be obtained
    return 1/(pi*(math.sqrt(math.sin(theta0/2)**2-math.sin(theta/2)**2)))

for theta0 in thetaList:
    runningavg = 0
    n = 4
    diff = 1
    while diff > 1E-5: # checks that the precision in the ratio is in the 5th decimal by comparing the integral for n points to the average of the integrals for n-1, n-2, and n-3 points
        n = n + 1 # increments number of points
        runningavg = 1/3*(gauss(partial(f,theta0),0,theta0,n-1)+gauss(partial(f,theta0),0,theta0,n-2)+gauss(partial(f,theta0),0,theta0,n-3)) # average of previous three results
        diff = gauss(partial(f,theta0),0,theta0,n)-runningavg # difference between current result and the average of the previous three
    print("The ratio T/T0 for theta0 = ", theta0, " is ", gauss(partial(f,theta0),0,theta0,n))


The ratio T/T0 for theta0 =  0.2617993877991494  is  1.0029051865219025
The ratio T/T0 for theta0 =  0.39269908169872414  is  1.008323802729877
The ratio T/T0 for theta0 =  0.5235987755982988  is  1.0159989871103667
The ratio T/T0 for theta0 =  0.7853981633974483  is  1.0385464791688406
The ratio T/T0 for theta0 =  1.0471975511965976  is  1.071723460737919
The ratio T/T0 for theta0 =  1.5707963267948966  is  1.1787835961947308
